In [ ]:
# from graph_original import *
from graph import *

import sympy
import symvec
from IPython.display import display
from sympy import *

#parameters of model
c1, c2, c3, u, D, kappa = sympy.symbols('c1 c2 c3 u D kappa')
c1t, c2t, c3t, ut, Dt, kappat = sympy.symbols('c1t c2t c3t ut Dt kappat')
alpha, a = 0, 0
# dimension and reduced surface area
d, Kd = sympy.symbols('d K_d')
# symbols for vectors
_q, _k, dot_kq = sympy.symbols('q k (k·q)')
_p, _k, dot_pk = sympy.symbols('p k (k·p)')
_q, _p, dot_qp = sympy.symbols('q p (q·p)')
_q, _r, dot_qr = sympy.symbols('q r (q·r)')
_p, _r, dot_pr = sympy.symbols('p r (p·r)')
_r, _k, dot_rk = sympy.symbols('r k (r·k)')

# assign symbol for dot product
symvec.dots[frozenset((_q,_k))] = dot_kq
symvec.dots[frozenset((_q,_p))] = dot_qp
symvec.dots[frozenset((_p,_k))] = dot_pk
symvec.dots[frozenset((_q,_r))] = dot_qr
symvec.dots[frozenset((_p,_r))] = dot_pr
symvec.dots[frozenset((_r,_k))] = dot_rk
# create vectors
k = symvec.Vector(_k)
q = symvec.Vector(_q)
p = symvec.Vector(_p)
r = symvec.Vector(_r)
def f(x, alpha, a):
    return (kappa*x**2+a)*x**alpha
# IMPORTANT: Simplify the vertex at the output
def v2(k1,k2,k3):
    expr = (c1*k3**2+2*c2*k1*k2-c3*(k2**2*k3*k1+k1**2*k3*k2)/(k3**2))*1/2
    return fraction(cancel((expr)))

def v2t(k1,k2,k3):
    expr = (c1t*k3**2+2*c2t*k1*k2-c3t*(k2**2*k3*k1+k1**2*k3*k2)/(k3**2))*1/2
    return fraction(cancel((expr)))

def v3(k1,k2,k3,k4):
    return (-u,1)


In [ ]:
#figure a
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[1].link_vertex(v[3])
v[2].link_vertex(v[3])
v[1].add_outgoing()
v[2].add_outgoing()

g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
g.label_edges(q,[p,q-p],k)

I = g.integral(f, D, k, alpha, a, v2, v3=0)
display(I)
I = symvec.integrate3(I, k, q, p, d, n=3)
display(simplify(I))


In [ ]:
# example: KPZ propagator
# construct graph with three vertices
v = [Vertex() for i in range(3)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[1].link_vertex(v[2])
v[1].add_outgoing()
g = Graph(v)
g.label_edges(q,[q],k)
I= g.integral(f, D, k, alpha, a, v2, v3)
display(I[0]/I[1])
g.plot_graph()
g.latex_graph('figure')


In [ ]:
from IPython.display import Image 
pil_img = Image(filename='./figures/modelB_plus_2vertex.jpg')
display(pil_img)

In [ ]:
#figure b
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[2].add_outgoing()
v[3].add_outgoing()

g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')

I_array = integrals_symmet(v, q, [p,q-p], f, D, k, alpha, a, v2, v3)
display(I_array[0])
display(I_array[1])

Ib = solve_integrals_symmet(I_array, k, [q, p], d, n=3)
display(simplify(Ib))

In [ ]:
#figure c
v = [Vertex() for i in range(3)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[1])
v[2].add_outgoing()
v[2].add_outgoing()

g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')

I_array = integrals_symmet(v, q, [p,q-p], f, D, k, alpha, a, v2, v3)
Ic = solve_integrals_symmet(I_array, k, [q, p], d, n=3)
display(Ic)
display(symvec.integrate_magnitude(Ic, k, d, q, p))



In [ ]:
#figure d
v = [Vertex() for i in range(3)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[1])
v[0].add_outgoing()
v[2].add_outgoing()

g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')

I_array = integrals_symmet(v, q, [p,q-p], f, D, k, alpha, a, v2, v3)
display(I_array[0])
Id = solve_integrals_symmet(I_array, k, [q, p], d, n=3)
display(Id)



In [ ]:
def expand_2vertex(expr, k, q, p):
    """
    Expands the coefficients of the renormalized 2-vertex
    Returns dictionary of the coefficients of the monomials
    """
    cs_psi, si_psi = sympy.symbols('cos_psi sin_psi')
    dot_qp = symvec.dots[frozenset((q.sym,p.sym))]
    expr = expr.subs(dot_qp,q.sym*p.sym*cs_psi)
    return symvec.all_coeffs(expr,[q.sym,p.sym,cs_psi])

def compare_coeffs(dict1,dict_original,k,q,p):
    """
    Returns equations between the renormalized model parameters and the original model parameters
    """
    cs_psi, si_psi = sympy.symbols('cos_psi sin_psi')
    eqts = [0]*len(dict1)
    i=0
    for key in dict1:
        eqts[i]= dict1.get(key) - dict_original.get(key)
        i+=1
    eqts = [i for i in eqts if i != 0]
    return eqts[1:]

def renormalize_2vertex(expr,k,q,p,d):
    """
    Matches the coefficients of the 2-vertex with the renormalized 2-vertex.
    """
    coeffs=symvec.func_coef(expr, q, p)
    coeffs_original = expand_2vertex(v2t(p,q-p,q),k,q,p)
    eqts= compare_coeffs(coeffs,coeffs_original,k,q,p)
    return list(linsolve(eqts, [c1t, c2t, c3t]))[0]
I=Poly(Ia+Ib,q.sym,p.sym).as_expr()
display(I)
sol_c1t, sol_c2t, sol_c3t = renormalize_2vertex(I,k,q,p,d)
display(simplify(sol_c1t))
display(simplify(sol_c2t))
display(simplify(sol_c3t))
c1ab=simplify(sol_c1t)
c2ab=simplify(sol_c2t)
c3ab=simplify(sol_c3t)

# Calculation of diagrams contributing to 3-vertex:

In [ ]:
from IPython.display import Image 
pil_img = Image(filename='./figures/modelB_plus_3vertex.jpg')
display(pil_img)

In [ ]:
import graph
from graph import *
import copy
import itertools

# figure e
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[1].link_vertex(v[2])
v[0].link_vertex(v[3])
v[3].link_vertex(v[2])
v[0].add_outgoing()
v[1].add_outgoing()
v[3].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]

Ie = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ie)

In [ ]:
import graph
from graph import *
import copy
import itertools

# figure f
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[0].add_outgoing()
v[2].add_outgoing()
v[3].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]

If = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(If)


In [ ]:
import graph
from graph import *
import copy
import itertools

# figure g
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[1].link_vertex(v[2])
v[0].link_vertex(v[3])
v[3].link_vertex(v[2])
v[1].add_outgoing()
v[1].add_outgoing()
v[3].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
display(I_array[2])
display(I_array[3])
display(I_array[4])
display(I_array[5])
print('---')
Ig = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ig)


In [ ]:
import graph
from graph import *
import copy
import itertools

# figure h
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[3].add_outgoing()
v[3].add_outgoing()
v[2].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
Ih = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ih)

In [ ]:
import graph
from graph import *
import copy
import itertools

# figure i
v = [Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[2].add_outgoing()
v[2].add_outgoing()
v[3].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
Ii = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ii)


In [ ]:
import graph
from graph import *
import copy
import itertools

# figure j
v = [Vertex() for i in range(5)]
v[0].link_vertex(v[1])
v[1].link_vertex(v[2])
v[2].link_vertex(v[3])
v[0].link_vertex(v[4])
v[4].link_vertex(v[3])
v[1].add_outgoing()
v[2].add_outgoing()
v[4].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
Ij = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ij)


In [ ]:
import graph
from graph import *
import copy
import itertools

# figure k
v = [Vertex() for i in range(5)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[4])
v[4].link_vertex(v[1])
v[2].add_outgoing()
v[3].add_outgoing()
v[4].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-r-p], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
Ik = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(Ik)


In [ ]:
import graph
from graph import *
import copy
import itertools

# figure 3c
v = [Vertex() for i in range(3)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[1])
v[0].add_outgoing()
v[2].add_outgoing()
v[2].add_outgoing()

I_array = []
g = Graph(v)
g.plot_graph()
g.latex_graph('graph_plot')
I_array = integrals_symmet(v, q, [p,r,q-p-r], f, D, k, alpha, a, v2, v3)
I_array = [(element[0].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)]), element[1].subs([(dot_qp,0), (dot_pk,0),(dot_qr,0), (dot_pr,0),(dot_rk,0),(p**2,0),(r**2,0)])) for element in I_array]
display(I_array[0])
display(I_array[1])
I3c = solve_integrals_symmet(I_array, k, [q, p], d, n=1)
display(I3c)


In [ ]:
Iefghi=Poly(Ie+If+Ig+Ih+Ii,q.sym).as_expr()
display(Iefghi)
Ijk=Poly(Ij+Ik,q.sym).as_expr()
display(Ijk)
I3c=Poly(I3c,q.sym).as_expr()
display(I3c)
display(simplify(Iefghi))
display(simplify(Ijk))
display(simplify(I3c))